### For given aoi, prepare TCI and NDVI sentinel latest images 

In [20]:
AOI = 'POLYGON ((35.4638671875000000 50.4365160169863316, 34.6618652343750000 50.0289165635219035, 34.7827148437500000 48.9549736980886792, 36.3757324218749929 48.3781454697624440, 37.4194335937500000 48.7851519980431547, 38.3038330078125000 49.4431287580300491, 37.9193115234374929 50.4120182466821731, 36.6009521484375000 50.5553249825196716, 35.4638671875000000 50.4365160169863316))' 
REQUEST_ID = 10
START_DATE = "2020-08-01"
END_DATE = "2020-08-10"

In [21]:
import os
import geopandas as gp
import numpy as np
import rasterio
import re
import tempfile
import pyproj
import uuid

import rasterio.mask
from rasterio import Affine
from rasterio.plot import reshape_as_raster
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling

from shapely import wkt
from shapely.geometry import Polygon, box
from shapely.ops import transform


from pathlib import Path
from datetime import datetime, timedelta
from sentinel2download.downloader import Sentinel2Downloader
from sentinel2download.overlap import Sentinel2Overlap

#### 1. Transform AOI and get bound_box

In [22]:
aoi = gp.GeoDataFrame(geometry=[wkt.loads(AOI)], crs="epsg:4326")    

In [23]:
aoi

,geometry
0,"POLYGON ((35.46387 50.43652, 34.66187 50.02892..."


#### Сreate base path, usually env=jovyan

In [24]:
BASE = f"/home/{os.getenv('NB_USER')}/work"
BASE

'/home/jovyan/work'

In [25]:
sentinel_grid = gp.read_file("sentinel2grid.geojson")

#### 2. Overlap AOI with sentinel2grid 

In [26]:
def epsg_code(longitude, latitude):
        """
        Generates EPSG code from lon, lat
        :param longitude: float
        :param latitude: float
        :return: int, EPSG code
        """

        def _zone_number(lat, lon):
            if 56 <= lat < 64 and 3 <= lon < 12:
                return 32
            if 72 <= lat <= 84 and lon >= 0:
                if lon < 9:
                    return 31
                elif lon < 21:
                    return 33
                elif lon < 33:
                    return 35
                elif lon < 42:
                    return 37

            return int((lon + 180) / 6) + 1

        zone = _zone_number(latitude, longitude)

        if latitude > 0:
            return 32600 + zone
        else:
            return 32700 + zone

In [27]:
def _intersect(aoi, grid):
    # Get the indices of the tiles that are likely to be inside the bounding box of the given Polygon
    geometry = aoi.geometry[0]

    tiles_indexes = list(grid.sindex.intersection(geometry.bounds))
    grid = grid.loc[tiles_indexes]

    # Make the precise tiles in Polygon query
    grid = grid.loc[grid.intersects(geometry)]

    # intersection area
    epsg = epsg_code(geometry.centroid.x, geometry.centroid.y)

    # to UTM projection in meters
    aoi.to_crs(epsg=epsg, inplace=True)
    grid.to_crs(epsg=epsg, inplace=True)

    return grid, epsg

In [28]:
def get_intersected_tiles(aoi, grid):
    
    grid, epsg = _intersect(aoi, sentinel_grid)
    
    
    grid.set_index("Name", drop=False, inplace=True)    
    intersected_grid = {"tile": [], "geometry": []}

    rest_aoi = aoi.copy()
    while rest_aoi.area.sum() > 0:
        intersection = gp.overlay(rest_aoi, grid, how="intersection")
        argmax = intersection.area.argmax()

        tile = intersection.loc[argmax, "Name"]
        intersected_geometry = intersection.loc[argmax, "geometry"]
        
        intersected_grid["tile"].append(tile)
        intersected_grid["geometry"].append(intersected_geometry)
        
        biggest_intersection = grid.loc[[tile]]
        rest_aoi = gp.overlay(rest_aoi, biggest_intersection, how="difference")
        grid = grid.loc[intersection["Name"]]
    
    overlap_tiles = gp.GeoDataFrame(intersected_grid, crs=epsg)
    overlap_tiles.to_crs(epsg=4326, inplace=True)

    return overlap_tiles

In [29]:
overlap_tiles = get_intersected_tiles(aoi.copy(), sentinel_grid.copy())
overlap_tiles

,tile,geometry
0,36UYA,"POLYGON Z ((35.81823 50.47479 0.00000, 36.1505..."
1,36UYV,"MULTIPOLYGON Z (((35.76362 49.53174 0.00000, 3..."
2,36UXV,"MULTIPOLYGON Z (((35.79074 48.63121 0.00000, 3..."
3,36UXA,"POLYGON Z ((35.46387 50.43652 0.00000, 35.8182..."
4,37UDR,"POLYGON Z ((37.59090 50.44920 0.00000, 37.9193..."
5,37UCQ,"POLYGON Z ((37.28127 49.52976 0.00000, 37.2910..."
6,37UCR,"POLYGON Z ((37.29103 49.64011 0.00000, 37.3657..."
7,37UDQ,"POLYGON Z ((37.75278 49.55803 0.00000, 38.2573..."
8,36UYU,"POLYGON Z ((35.71416 48.63304 0.00000, 35.7907..."
9,36UYB,"MULTIPOLYGON Z (((36.24884 50.51978 0.00000, 3..."


In [30]:
overlap_tiles.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

#### 3. Load images

In [31]:
API_KEY = os.path.join(BASE, ".secret/sentinel2_google_api_key.json")
LOAD_DIR = os.path.join(BASE, "satellite_imagery")

PRODUCT_TYPE = 'L2A'
BANDS = {'TCI', 'B04', 'B08', }
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 10.0, 'CLOUDY_PIXEL_PERCENTAGE': 5.0, }

LAYERS = ['TCI', 'NDVI', ]

In [32]:
START_DATE

'2020-08-01'

In [33]:
END_DATE

'2020-08-10'

In [34]:
def shift_date(date, delta=5, format='%Y-%m-%d'):
    date = datetime.strptime(date, format)
    date = date - timedelta(days=delta)    
    return datetime.strftime(date, format)

#### 3.1 Define max shift in dates - 30 days for loading images

In [43]:
MAX_SHIFT = 30

In [44]:
MAX_SHIFT_ITERS = 2

In [45]:
def load_images(tiles, start_date, end_date):
    loader = Sentinel2Downloader(API_KEY)
    
    loadings = dict()
        
    for tile in tiles:
        start = start_date
        end = end_date
        
        print(f"Loading images for tile: {tile}...")
        count = 0
        while count < MAX_SHIFT_ITERS:
            loaded = loader.download(PRODUCT_TYPE,
                                [tile],
                                start_date=start,
                                end_date=end,
                                output_dir=LOAD_DIR,                       
                                bands=BANDS,
                                constraints=CONSTRAINTS)
        
            if not loaded:
                end = start_date
                start = shift_date(start_date, delta=MAX_SHIFT) 
                print(f"For tile: {tile} and dates {start_date} {end_date} proper images not found! Shift dates to {start} {end}!")
            else:
                break
            count += 1
        if loaded:
            loadings[tile] = loaded
            print(f"Loading images for tile {tile} finished")
        else:
            print(f"Images for tile {tile} were not loaded!")
        
    # tile_folders = dict()
    # for tile, tile_paths in loadings.items():
    #    tile_folders[tile] = {str(Path(tile_path[0]).parent) for tile_path in tile_paths}
    return loadings

In [46]:
loadings = load_images(overlap_tiles.tile.values, START_DATE, END_DATE)

Loading images for tile: 36UYA...
For tile: 36UYA and dates 2020-07-02 2020-08-01 proper images not found! Shift dates to 2020-07-02 2020-08-01!
Loading images for tile 36UYA finished
Loading images for tile: 36UYV...
Loading images for tile 36UYV finished
Loading images for tile: 36UXV...
Loading images for tile 36UXV finished
Loading images for tile: 36UXA...
Loading images for tile 36UXA finished
Loading images for tile: 37UDR...
Loading images for tile 37UDR finished
Loading images for tile: 37UCQ...
Loading images for tile 37UCQ finished
Loading images for tile: 37UCR...
For tile: 37UCR and dates 2020-07-02 2020-08-01 proper images not found! Shift dates to 2020-07-02 2020-08-01!
Loading images for tile 37UCR finished
Loading images for tile: 37UDQ...
Loading images for tile 37UDQ finished
Loading images for tile: 36UYU...
Loading images for tile 36UYU finished
Loading images for tile: 36UYB...
Loading images for tile 36UYB finished
Loading images for tile: 36UXU...
Loading images

#### 3.2 Filter loadings for every tile, get last image in daterange and bands

In [47]:
def filter_by_date(loadings):
    def _find_last_date(folders):        
        dates = list()
        for folder in folders:        
            search = re.search(r"_(\d+)T\d+_", str(folder))
            date = search.group(1)
            date = datetime.strptime(date, '%Y%m%d')
            dates.append(date)    
        last_date = max(dates)
        last_date = datetime.strftime(last_date, '%Y%m%d')
        return last_date
    
    filtered = dict()
    for tile, items in loadings.items():
        try:
            last_date = _find_last_date(items)
            bands_paths = dict()
            for path, _ in items:
                if last_date in path:
                    if 'B04_10m.jp2' in path:
                        bands_paths['RED'] = path
                    if 'B08_10m.jp2' in path:
                        bands_paths['NIR'] = path
                    if 'TCI_10m.jp2' in path:
                        bands_paths['TCI'] = path
            filtered[tile] = dict(paths=bands_paths, date=last_date)
        except Exception as ex:
            print(f"Error for {tile}: {str(ex)}")
    return filtered

In [51]:
filtered = filter_by_date(loadings)

In [54]:
if not filtered:
    raise ValueError("Images not loaded for given AOI. Change dates, constraints")

#### 4. Calculate NDVI

In [55]:
RESULTS_DIR = os.path.join(BASE, "results/example/tci_ndvi")

NOTEBOOK_DIR = os.path.join(BASE, "notebooks/example/tci_ndvi")
COLORMAP_BRBG = os.path.join(NOTEBOOK_DIR, "brbg.npy") 

#### 4.1 Prepare color coding for NDVI

In [56]:
def prepare_colors(colors):
    colors = np.load(COLORMAP_BRBG)
    # delete last channel, we use rgb
    colors = np.delete(colors, 3, axis=1)
    # colormap colors values in range [0-255], but in our case 0 - no data, -> have to color as [0, 0, 0] 
    colors[colors == 0] = 1
    colors[0] = [0, 0, 0]
    return colors

In [57]:
COLORS = prepare_colors(COLORMAP_BRBG)

In [58]:
COLORS.shape

(256, 3)

In [62]:
def scale(ndvi, a=1, b=255, nodata=0.0):
    # ndvi is in range [-1; 1], nodata is setted to 0.0 value. Be careful with comprassions!
    min = -1 # np.nanmin(ndvi)
    max = 1 # np.nanmax(ndvi)
    scaled = (b - a) * (ndvi - min) / (max - min) + a
    scaled = np.around(scaled)
    scaled[np.isnan(scaled) == True] = nodata
    scaled = scaled.astype(np.uint8)
    return scaled

In [63]:
def color_ndvi(scaled, colors):
    colored = np.reshape(colors[scaled.flatten()], tuple((*scaled.shape, 3)))
    colored = reshape_as_raster(colored)
    return colored

In [64]:
def NDVI(nir_path, red_path, save_path):
    # Asllow division by zero
    np.seterr(divide='ignore', invalid='ignore')
    
    with rasterio.open(nir_path) as src:
        nir = src.read(1).astype(rasterio.float32)
        crs = str(src.crs)
    with rasterio.open(red_path) as src:
        red = src.read(1).astype(rasterio.float32)

    # Calculate NDVI
    ndvi = ((nir - red) / (nir + red)) 
    
    scaled = scale(ndvi)
    colored = color_ndvi(scaled, COLORS) 
    
    
    # Set spatial characteristics of the output object
    out_meta = src.meta.copy()    
    out_meta.update(dtype=rasterio.uint8,
                    driver='GTiff',
                    nodata=0,
                    count=3, )

    # Create the file
    with rasterio.open(save_path, 'w', **out_meta) as dst:
         dst.write(colored)
    return crs

In [65]:
def to_crs(poly, target, current='EPSG:4326'):
    # print(f"TARGET CRS: {target}")
    project = pyproj.Transformer.from_crs(pyproj.CRS(current), pyproj.CRS(target), always_xy=True).transform
    transformed_poly = transform(project, poly)
    return transformed_poly 

In [66]:
def crop(input_path, output_path, polygon, date, request_id, name=None):
    with rasterio.open(input_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, [polygon], crop=True)
        # print(out_transform)
        out_meta = src.meta
        
        out_meta.update(driver='GTiff',
                        height=out_image.shape[1],
                        width=out_image.shape[2],
                        transform=out_transform,
                        nodata=0, )

    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.update_tags(start_date=date, end_date=date, request_id=request_id)
        if name:
            dest.update_tags(name=name)
        dest.write(out_image)

#### 4.2 Calculate and crop NDVI, TCI

#### Filenames have next names: REQUESTID_TILE_ID_ACQUIREDDATE

In [67]:
RESULTS_DIR

'/home/jovyan/work/results/example/tci_ndvi'

In [68]:
RESULTS_DIR = os.path.join(RESULTS_DIR, str(REQUEST_ID))
RESULTS_DIR

'/home/jovyan/work/results/example/tci_ndvi/10'

In [69]:
RESULTS_DIR = '/home/jovyan/work/notebooks/example/tci_ndvi/10'

In [70]:
os.makedirs(RESULTS_DIR, exist_ok=True)

In [88]:
for row in overlap_tiles.itertuples():
    tile = row.tile
    polygon = row.geometry
    
    try:
        paths = filtered[tile]['paths']
        print(f"{tile}: Start calculation TCI, NDVI")
        
        acquired_date = filtered[tile]['date']
        base_filename = f"{REQUEST_ID}_{tile}_{acquired_date}_"
        temp_ndvi_filename = os.path.join(RESULTS_DIR, base_filename + "NDVI.tif.temp")
        temp_tci_filename = os.path.join(RESULTS_DIR, base_filename + "TCI.tif.temp")
        
        tile_crs = NDVI(paths['NIR'], paths['RED'], temp_ndvi_filename)
        transformed_poly = to_crs(polygon, tile_crs)
        
        # Crop and save NDVI
        crop(temp_ndvi_filename, temp_ndvi_filename, transformed_poly, acquired_date, REQUEST_ID)
        # Crop and save TCI
        crop(paths['TCI'], temp_tci_filename, transformed_poly, acquired_date, REQUEST_ID)
        
        print(f"{tile}: End calculation TCI, NDVI")
    
        ndvi_filename = temp_ndvi_filename[:-5]
        tci_filename = temp_tci_filename[:-5]
        print(f"{tile}: Rename {temp_ndvi_filename}->{ndvi_filename}\n {temp_tci_filename}->{tci_filename}")
        os.rename(temp_ndvi_filename, ndvi_filename)
        os.rename(temp_tci_filename, tci_filename)
        
    except Exception as e:
        print(f"{tile}: Cannot calculate TCI, NDVI: {str(e)}")

36UYA: Start calculation TCI, NDVI
36UYA: End calculation TCI, NDVI
36UYA: Rename /home/jovyan/work/notebooks/example/tci_ndvi/10/10_36UYA_20200705_NDVI.tif.temp->/home/jovyan/work/notebooks/example/tci_ndvi/10/10_36UYA_20200705_NDVI.tif
 /home/jovyan/work/notebooks/example/tci_ndvi/10/10_36UYA_20200705_TCI.tif.temp->/home/jovyan/work/notebooks/example/tci_ndvi/10/10_36UYA_20200705_TCI.tif
36UYV: Start calculation TCI, NDVI
36UYV: End calculation TCI, NDVI
36UYV: Rename /home/jovyan/work/notebooks/example/tci_ndvi/10/10_36UYV_20200804_NDVI.tif.temp->/home/jovyan/work/notebooks/example/tci_ndvi/10/10_36UYV_20200804_NDVI.tif
 /home/jovyan/work/notebooks/example/tci_ndvi/10/10_36UYV_20200804_TCI.tif.temp->/home/jovyan/work/notebooks/example/tci_ndvi/10/10_36UYV_20200804_TCI.tif
36UXV: Start calculation TCI, NDVI
36UXV: End calculation TCI, NDVI
36UXV: Rename /home/jovyan/work/notebooks/example/tci_ndvi/10/10_36UXV_20200804_NDVI.tif.temp->/home/jovyan/work/notebooks/example/tci_ndvi/10/10_